In [1]:
import cv2
import numpy as np

# 读取图像
image = cv2.imread('dog.jpg',cv2.IMREAD_GRAYSCALE)

# 应用高斯滤波
sigma1 = 1
sigma2 = 2
blurred_image1 = cv2.GaussianBlur(image, (5, 5), sigma1)
blurred_image2 = cv2.GaussianBlur(image, (5, 5), sigma2)
DoG = blurred_image2 - blurred_image1


# 显示结果
cv2.imshow('Original', image)
cv2.imshow('Blurred', DoG)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import cv2

In [6]:
class DoGFilter(nn.Module):
    def __init__(self, in_channels, kernel_size,sigma1,sigma2, stride=1, padding=0):
        super(DoGFilter, self).__init__()
        self.in_channels = in_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.sigma1 = sigma1
        self.sigma2 = sigma2
        # initiate
        self.weight1 = nn.Parameter(torch.randn(1, in_channels, kernel_size, kernel_size))
        self.weight2 = nn.Parameter(torch.randn(1, in_channels, kernel_size, kernel_size))
        
        #create gaussin kernel 
    def DoG_kernel(self, sigma1, sigma2, size):
        size = int(size) // 2
        x, y = np.mgrid[-size:size+1, -size:size+1]
        normal1 = 1 / (2.0 * np.pi * sigma1**2)
        normal2 = 1 / (2.0 * np.pi * sigma2**2)
        g1 = np.exp(-((x**2 + y**2) / (2.0*sigma1**2))) * normal1
        g2 = np.exp(-((x**2 + y**2) / (2.0*sigma2**2))) * normal2
        # transfer to Tensor
        return torch.from_numpy(g1).float(), torch.from_numpy(g2).float()   

         
        
    def forward(self, x, sigma1, sigma2):
        # create gaussin kernel
        self.weight1, self.weight2 = self.DoG_kernel(self.sigma1, self.sigma2, self.kernel_size)
        x1 = F.conv2d(x, self.weight1, self.bias, self.stride, self.padding)
        x2 = F.conv2d(x, self.weight2, self.bias, self.stride, self.padding)
        x_on = x1 - x2 #on center filter
        x_off = x2 - x1#off center filter
        x = torch.cat((x_on, x_off), dim=1)
        return x
          

In [75]:
def DoG_kernel1(size=5,s1=1,s2=2):
        r = np.arange(size)+1
        x = np.tile(r, [size, 1])
        y = x.T
        d2 = (x-size/2.-0.5)**2 + (y-size/2.-0.5)**2
        filt = 1/np.sqrt(2*np.pi) * (1/s1 * np.exp(-d2/(2*(s1**2))) - 1/s2 * np.exp(-d2/(2*(s2**2))))
        filt -= np.mean(filt[:])
        filt /= np.amax(filt[:])  
        return filt 

In [78]:
image = cv2.imread('dog.jpg',cv2.IMREAD_GRAYSCALE)
dogfilter = DoGFilter(in_channels=1, sigma1=1,sigma2=2,kernel_size=5)
filter = DoG_kernel1(size=5,s1=1,s2=2)
image = torch.from_numpy(image)
filter = torch.from_numpy(filter)
image = image.view(1,1000,1500).float()
filter = filter.view(1,1,5,5).float()
conv2d  = nn.Conv2d(1,1,(5,5))
conv2d.weight = nn.Parameter(filter,requires_grad = False)


weight1 torch.Size([1, 1, 5, 5])
weight2 torch.Size([1, 1, 5, 5])
type is <class 'torch.Tensor'>


In [83]:
x1 = conv2d(image)
x1 = x1.detach().numpy()
print(x1)

[[[ -0.80040765  -0.80040765  -0.80040765 ...   0.48368493   0.8552364
     0.8552364 ]
  [ -0.62512994  -0.62512994  -0.62512994 ...  -0.25646394   0.11508758
     0.11508758]
  [  0.855232     0.855232     0.855232   ...  -0.79744905  -0.62510103
    -0.62510103]
  ...
  [-11.791835   -12.618955   -11.510163   ...  -2.2406344   -4.3663735
    -3.8217351 ]
  [-11.664504    -5.8709316   -4.0149918  ...  -2.7627463   -5.1355886
    -3.7408552 ]
  [-11.163313     3.001029     5.3249264  ...   0.2954594   -2.7611127
    -0.6025919 ]]]


: 

In [4]:
from scipy.ndimage import gaussian_filter
import cv2
import numpy as np
image = cv2.imread('dog.jpg',cv2.IMREAD_GRAYSCALE)
x1 = gaussian_filter(image,1)
x2 = gaussian_filter(image,2)
x_on = x1 - x2 #on center filter 
x_off = x2 - x1#off center filter
np.printoptions(threshold=np.inf)
cv2.imshow("on filter",x_on)
cv2.imshow("off filter",x_off)
cv2.waitKey(0)
cv2.destroyAllWindows
print("x_on",x_on[200:500,200:600])

x_on [[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 1  1  1 ... 14 15 17]
 [ 0  0  0 ... 20 22 25]
 [ 0  0  0 ... 16 19 22]]


In [8]:
from DoGFilter_copy import DoGFilter
import cv2
image = cv2.imread('dog.jpg',cv2.IMREAD_GRAYSCALE)
filter = DoGFilter(in_channels=1,kernel_size=5,sigma1=1,sigma2=2)
filter_result = filter.forward(image)
filter_result = filter_result.numpy()
cv2.imshow("off filter",filter_result)
cv2.waitKey(0)
cv2.destroyAllWindows

<function destroyAllWindows>